In [4]:
%load_ext autoreload
%autoreload 2

!pip install -e ../final-assignment/

import pandas as pd

Obtaining file:///C:/Users/PiotrGrabysz/Documents/PythonScripts/NYPD/final-assignment
  Attempting uninstall: Warsaw-buses
    Found existing installation: Warsaw-buses 0.0.1
    Uninstalling Warsaw-buses-0.0.1:
      Successfully uninstalled Warsaw-buses-0.0.1
  Running setup.py develop for Warsaw-buses


In [5]:
from pathlib import Path

dir_to_busestrams = "../final_assignment_data/busestrams_05_02"

## Filtering the data

First of all, you should filter the data. I found that sometimes the same brigade of the same line has a few timestamps describing **the same time**. Or its timestamps are not sorted in icreasing order. You can see this situation here:

In [15]:
buses_data1 = pd.read_csv(Path(dir_to_busestrams) / "115/2/1413.txt", names=["Lat", "Lon", "Time"])
buses_data1.head(10)

,Lat,Lon,Time
0,52.248985,21.178604,2021-02-05 20:23:09
1,52.252907,21.167728,2021-02-05 20:25:08
2,52.249443,21.177103,2021-02-05 20:23:19
3,52.249657,21.176409,2021-02-05 20:23:24
4,52.250217,21.174595,2021-02-05 20:23:39
5,52.255165,21.160923,2021-02-05 20:26:24
6,52.255329,21.159365,2021-02-05 20:26:36
7,52.255371,21.158566,2021-02-05 20:26:45
8,52.255383,21.158396,2021-02-05 20:26:56
9,52.250366,21.174135,2021-02-05 20:23:44


As you can see, row 2 is earlier than row 1 and row 9 is earlier than row 5, 6, 7, 8. You can check the whole data set for such problems.

In [17]:
from Warsaw_buses.process_data import filter_data

filter_data.is_time_monotonically_increasing(dir_to_data=dir_to_busestrams)

Subsequent measurements are not time monotonous!. I've found 243 incorrect files which is 18% of the total number of files. See C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\is_time_monotonically_increasing_errors_log.txt for details.


One can take two approaches here. The first one is to sort the files by their timestamps. The second one is to filter out this files from further analysis.
Because majority of files is sorted as it is downloaded from api I treat this unsorted files suspiciously and decided to filter them out. 

In [39]:
filter_data.filter_data_from_non_monotonically_increasing_time(dir_to_data=dir_to_busestrams)

dir_to_busestrams += "_filtered"
print(f"The correct files are saved in the {Path(dir_to_busestrams).absolute()}")

The correct files are saved in the C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\..\final_assignment_data\busestrams_05_02_filtered


## Speed analysis

#### Calculating the speed from data

Firstly you need to calculate the speed of buses. I decided to create a new directory instead of extending the existing directory dir_to_busestrams. The new directory's name is dir_to_busestrams suffixed with "_with_speed". It contains all the files from dir_to_busestrams with an additional column standing for the speed. 

To calculate the speed, please run run_data_analysis with a flag --calc_speed. 

You should also provide the directory to the busestrams files with a parameter 

> --dir_to_busestrams DIR_TO_BUSESTRAMS, -db DIR_TO_BUSESTRAMS

                        The directory with the busestrams data.

In [56]:
! run_data_analysis --calc_speed -db ../final_assignment_data/busestrams_05_02_filtered

Using buses data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\..\final_assignment_data\busestrams_05_02_filtered
Using timetables data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\data\timetables
Using coordinates of the stops data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\data\timetables\stops_coord.json

Starts calculating speed...
Speed calculated and saved successfully.


#### Speed statistiscs
Once we have the speed, we can analyse it. Use a flag --speed_statistics to see some statistics. There are two parameters here:

> --exceeded_speed EXCEEDED_SPEED, -s EXCEEDED_SPEED

                        The program looks for the times when the speed
                        specified here is exceeded. It is set to 50 km/h by default.


> --outlier_speed OUTLIER_SPEED

                        If specified, speeds greater than outlier_speed are
                        not taken into consideration. It is set to 120
                        km/h by default.
                      
Why is outlier_speed needed? Well, take a look at the line 523 brigade 016:

In [66]:
buses_data = pd.read_csv(Path(dir_to_busestrams+"_with_speed") / "523/016/2006.txt", index_col=0)
print(buses_data[55:60])

          Lat        Lon                 Time        Speed
56  52.235382  21.078302  2021-02-05 16:26:39    28.138983
57  52.196430  20.921320  2021-02-05 16:27:47   611.058660
58  52.229946  21.067240  2021-02-05 16:27:59  3186.212607
59  52.228474  21.061771  2021-02-05 16:28:41    34.884285
60  52.196423  20.921326  2021-02-05 16:29:19   967.669224


The speed is in kilometers per hour so the speed of 3186 km/h is measured here! But it is not the bug of the algorithm: the distance between bus positions in rows 57 and 58 is **more than 10 kilometers**! And according to the data this distance was passed in 12 seconds.

In [35]:
from Warsaw_buses.process_data.utils import dist

print(f"The distance between (52.19643, 20.92132) and (52.229946, 21.06724) is {dist(52.19643, 20.92132, 52.229946, 21.06724):.2f} km.")

The distance between (52.19643, 20.92132) and (52.229946, 21.06724) is 10.62 km.


That is why I decided to set some threshold called outlier_speed. If a speed greater than the outlier_speed is measured it is just not considered. I set this to 120 km/h by default, but you can change it to whatever you want. 

The program reports if the speed greater than outlier speed was found anywhere. 

In [67]:
! run_data_analysis --speed_statistics \
--outlier_speed 120 \
-db ../final_assignment_data/busestrams_05_02_filtered

Using buses data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\..\final_assignment_data\busestrams_05_02_filtered
Using timetables data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\data\timetables
Using coordinates of the stops data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\data\timetables\stops_coord.json

Starts calculating statistics...
Warning! I found a bus with speed faster than 120.0 in the file ..\final_assignment_data\busestrams_05_02_filtered_with_speed\102\2\9422.txt
Warning! I found a bus with speed faster than 120.0 in the file ..\final_assignment_data\busestrams_05_02_filtered_with_speed\105\1\5228.txt
Warning! I found a bus with speed faster than 120.0 in the file ..\final_assignment_data\busestrams_05_02_filtered_with_speed\106\1\1552.txt
Warning! I found a bus with speed faster than 120.0 i

#### Locations with significant percent of buses exceeding the speed

Run the command run_data_analysis with a flag --speed_locations and these parameters:

> --exceeded_speed EXCEEDED_SPEED, -s EXCEEDED_SPEED

                        The program looks for the times when the speed
                        specified here is exceeded. It is set to 50 km/h by
                        default.It is used in speed_statistics() and
                        exceeding_the_speed_location().
                        
>  --outlier_speed OUTLIER_SPEED
  
                        If specified, speeds greater than outlier_speed are
                        not taken into consideration. It is set to 120 km/h by
                        default.
                        
>  --round_to ROUND_TO   
  
                        The notion of an exceeding location is rounded up to
                        round_to decimal places. It is set to 2 by default.
                        

The program looks for every point where the bus speed is greater than the speed specified by the user (but smaller than the outlier_speed). Then the latitude and longitude of this point is rounded up to round_to decimal places which means that the Warsaw map is devided into rectangles.

The program returns five locations where the ratio of measurements with speed greater than the exceeding_speed to number of all measurements taken in this locations is highest.

In [46]:
! run_data_analysis --speed_locations -db ../final_assignment_data/busestrams_05_02_filtered

Using buses data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\..\final_assignment_data\busestrams_05_02_filtered
Using timetables data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\data\timetables
Using coordinates of the stops data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\data\timetables\stops_coord.json

Looking for locations were 50.0 km/h is exceeded...
You chose round_to as 2 so the map is divided into rectangles of the size 1.112 km height and 0.685 km width.

Five locations with the biggest percent of buses exceeding the speed:
lat    lon      exceeding %
52.17, 21.09    100% 
52.11, 20.95    100% 
52.1, 20.95    100% 
52.05, 20.96    100% 
52.14, 21.09    100% 


As you can see, in all of those places all the buses run with the speed greater than 50km/h all the time. But these locations lies at the outskirts of Warsaw (more precisely they lie in Błonia Wilanowskie, Lesznowola and Piaseczno) so it is likely that the speed restriction is higher than 50 km/h and the the stops are spread more rarely, so the buses don't have to stop often.

So I come up with an idea to pay attention only to the locations lying in (more or less) centre of Warsaw. The user can define the parameter --radius. Only the locations lying in a radius smaller than radius from the point [52.23213243754829, 21.0060709762026] (the Palace of Science and Culture) are considered.

In [47]:
! run_data_analysis --speed_locations \
-db ../final_assignment_data/busestrams_05_02_filtered \
--radius 8.

Using buses data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\..\final_assignment_data\busestrams_05_02_filtered
Using timetables data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\data\timetables
Using coordinates of the stops data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\data\timetables\stops_coord.json

Looking for locations were 50.0 km/h is exceeded...
You chose round_to as 2 so the map is divided into rectangles of the size 1.112 km height and 0.685 km width.

Five locations with the biggest percent of buses exceeding the speed:
lat    lon      exceeding %
52.19, 20.92    81% 
52.29, 20.99    73% 
52.21, 21.09    71% 
52.3, 21.0    60% 
52.2, 21.06    57% 


81% still seem to be high. But the location 52.19, 20.92 lies at the Trasa Salomea-Wolica, where the cars drive very fast.

#### Visualization on the map

The much easier way to talk about places where the speed is exceeded is to show those places on the map. I found that folium package can to this. 

run_data_analysis --speed_locations apart from printing some info also save the locations and the corresponding number of exceeding times into binary file "./speed_exceeded_pickle". It can be loaded again and used by the folium package. Please, specify the threshold to show the locations where the exceeded % is greater than the threshold.

In [37]:
# !pip install folium

import folium
import pickle

thres = 0.3
radius = 10.

In [44]:
with open("./speed_exceeded_pickle", "rb") as f:
    exceeding_locations_dict = pickle.load(f)
    
Warsaw_centre = [52.23213243754829, 21.0060709762026]
Warsaw_map=folium.Map(location=Warsaw_centre, zoom_start = 11)
folium.Marker(location=Warsaw_centre, tooltip="centre",
              icon=folium.Icon(color='red')).add_to(Warsaw_map)

for loc, counts in exceeding_locations_dict.items():
    if dist(Warsaw_centre[0], Warsaw_centre[1], loc[0], loc[1]) < radius:
        exceeding_percent = counts[1] / counts[0]
        if exceeding_percent > thres:
            folium.Marker(location=[loc[0], loc[1]],
                          tooltip=f"{loc} : {100*exceeding_percent:.0f}%").add_to(Warsaw_map)
# Show the map
Warsaw_map

The map above shows all the locations up to 10 km from Warsaw centre where to percent of exceeding times is greater than 30%. As you can see these places lie in Warsaw surroundings near the fast ways like S8 or S2 which is reasonable.

## Analysis of buses punctuality

After dowloading the timetables as described in "collect_data sample usage.ipynb" I have an access to when and where each **brigade** should arrive. I could analyze if any bus of the given line has arrived at the stop at the time it should which means there could be no observed delay if the bus was so delayed that it land in the schedule of its subsequent bus.

I want to take another approach and anylyze the delays "brigade-wise". To see when the brigade arrived at the stop I would like to find the timestamp when the the brigade was closest the this stop. But if during the time of collecting the data this brigade went back to the final stop **and returned**, then it might passed this stop twice. To ensure that the timestamp of the closest timestamp is the right one I redefine the notion of "closeness" and add some time penalty to it. Let $x_{bus}$ be the bus position and $t_{bus}$ the corresponding timestamp and $x_{stop}$ the position of the stop, $t_{stop}$ the time when the bus should arrive at this stop. Then the distance between the bus and the stop is:

\begin{equation}
dist_{new} = dist(x_{bus}, x_{stop}) + \alpha \cdot |t_{bus} - t_{stop}|
\end{equation}

Where $dist()$ is the normal distance in space (I use the haversine distance) and $\alpha$ is a weight describing the time difference impact (it is set to 0.001 by default).

The idea to calculate the delay of each brigade is as follows:

+ get the brigade's schedule: position of each stop the brigade goes to and the time when this brigade should arrive at this stop

+ remove from the schedule rows with time of arrival earlier than the earliest time brigade's timestamp and later then the latest brigade's timestamp (if you collect the buses data from 15:00 to 16:00, consider only those in the schedule)

+ for each stop find the bus $(x_{bus}^{*}, t_{bus}^{*})$ minimizing the $dist_{new}$ from the stop. Once this position is found, estimate the time needed to reach the stop from this position as follows:
    \begin{equation}
        delay = t_{bus}^{*} - t_{stop} + \frac{dist(x_{bus}^{*}, x_{stop})}{v_{max}}
    \end{equation}
    
    $\frac{dist(x_{bus}^{*}, x_{stop})}{v_{max}}$ estimats the time needed the reach the stop (at location $x_{stop}$ from the bus position $x_{bus}$. $v_{max}$ is set to 40 km/h meaning that I assume that the bus goes with 40 km/h speed around the stop. It is just an naive assumption but I wanted to tackle somehow (the often) situation when the bus reports its position closely to the stop, but not excactly at the stop.

To do punctuality analysis, please run the command run_data_analysis with the flat --punctuality and these parameters:

> --dir_to_busestrams DIR_TO_BUSESTRAMS, -db DIR_TO_BUSESTRAMS
                    
                    The directory with the busestrams data.
> --dir_to_timetables DIR_TO_TIMETABLES, -dt DIR_TO_TIMETABLES
                    
                    The directory where the files with timetables are
                    stored.
> --dir_to_stops_coord DIR_TO_STOPS_COORD, -ds DIR_TO_STOPS_COORD
                    
                    The name of the file with bus stops coordinates.


In [57]:
! run_data_analysis --punctuality \
-db ../final_assignment_data/busestrams_05_02_filtered \
-dt ../final_assignment_data/timetables_05_02 \
-ds ../final_assignment_data/timetables_05_02/stops_coord.json

Using buses data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\..\final_assignment_data\busestrams_05_02_filtered
Using timetables data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\..\final_assignment_data\timetables_05_02
Using coordinates of the stops data from the directory C:\Users\PiotrGrabysz\Documents\PythonScripts\NYPD\final-assignment_test_installation\..\final_assignment_data\timetables_05_02\stops_coord.json

Starts calculating delays...
It might take a while...

Delays statistics:
--------------------
Mean delay is: 0 minutes and 50 seconds.
Percent of times when the bus is on time or its delay is at most one minute: 40%
Percent of delays longer than one minute: 59%.
Percent of delays longer than five minute: 55%.
The longest found delay is: 37 minutes and 38 seconds.
--------------------
